In [397]:
pip install nltk pymorphy3

In [399]:
from typing import List, Dict
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import math
import numpy as np

In [401]:
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Создаем класс с методами обработки текста для дальнейшей работы

In [404]:
class Text_transformation:
    def tokenize(self, text: str) -> List[str]:
        #токенизация текста
        return word_tokenize(text)

    def lemmas(self, tokens: List[str]) -> List[str]:
        #лемматизация полученных токенов
        morph3 = MorphAnalyzer()
        return [morph3.parse(word)[0].normal_form for word in tokens]

    def stemmas(self, tokens: List[str]) -> List[str]:
        #стемминг к токенам
        stemmer = SnowballStemmer("russian")
        return [stemmer.stem(word) for word in tokens]

    def vectors(self, tokens: List[str]) -> List[int]:
        #приведение токенов в ниукальные векторные представления
        token_to_dict_vectors = {word: idx for idx, word in enumerate(set(tokens))}
        return [token_to_dict_vectors[word] for word in tokens]

    def vectors_in_dict(self, tokens: List[str]) -> Dict[str, int]:
        #преобразование векторов в словарь с уникальными индексами
        return {word: i for i, word in enumerate(set(tokens))}

    def remove_stop_words(self, tokens: List[str]) -> List[str]:
        #Удаление стоп-слов из списка токенов.
        stop_words = set(stopwords.words('russian')).union({'.', ',', ':', '?', '!', '—'})
        return [word for word in tokens if word not in stop_words]

    def BoW(self, tokens: List[str]) -> Dict[str, int]:
        count = {}
        for word in tokens:
            count[word] = count.get(word, 0) + 1
        return count

    def tf(self, tokens: List[str]) -> Dict[str, float]:
        #Вычисление TF
        word_count = self.bag_of_words(tokens)
        total_words = len(tokens)
        return {word: count / total_words for word, count in word_count.items()}

    def idf(self, texts: List[List[str]]) -> Dict[str, float]:
        #Вычисление IDF
        total_texts = len(texts)
        all_words = set(word for text in texts for word in set(text))
        return {word: math.log(total_texts / sum(word in text for text in texts)) for word in all_words}

    def tf_idf(self, texts: List[List[str]], indexText: int) -> Dict[str, float]:
        #Вычисление TF-IDF
        tf = self.tf(texts[indexText])
        idf = self.idf(texts)
        return {word: tf[word] * idf.get(word, 0) for word in tf}

In [661]:
class Transformer:
    def __init__(self, size, embed, hidden, lr=0.001):
        
        self.embed = embed
        self.hidden = hidden
        self.size = size
        self.lr = lr

        self.embeddings = np.random.randn(size, embed) * 0.01

        self.q_SA = np.random.randn(embed, hidden) * 0.01
        self.k_SA = np.random.randn(embed, hidden) * 0.01
        self.v_SA = np.random.randn(embed, hidden) * 0.01

        self.outt = np.random.randn(hidden, size) * 0.01

    def softmax(self, x):
        exp = np.exp(x - np.max(x))
        return exp / np.sum(exp)

    def forward(self, input_indices):
        self.input_indices = input_indices
        self.x = self.embeddings[input_indices]

        self.Q = np.dot(self.x, self.q_SA)
        self.K = np.dot(self.x, self.k_SA)
        self.V = np.dot(self.x, self.v_SA)

        self.scores = np.dot(self.Q, self.K.T) / np.sqrt(self.embed)
        self.weights = self.softmax(self.scores)

        self.attended = np.dot(self.weights, self.V)
        self.context = np.mean(self.attended, axis=0)
    
        self.logits = np.dot(self.context, self.outt)
        self.probs = self.softmax(self.logits)

        return self.probs

    def backward(self, target_index):
        dlogits = self.probs.copy()
        dlogits[target_index] -= 1
        dW_out = np.outer(self.context, dlogits)

        dcontext = np.dot(dlogits, self.outt.T)

        dattended = np.ones_like(self.attended) * dcontext / self.attended.shape[0]
    
        dV = np.dot(self.weights.T, dattended)
        d_attn_weights = np.dot(dattended, self.V.T)
    
        d_scores = d_attn_weights * self.weights * (1 - self.weights)
    
        dQ = np.dot(d_scores, self.K) / np.sqrt(self.embed)
        dK = np.dot(d_scores.T, self.Q) / np.sqrt(self.embed)
    
        dW_q = np.dot(self.x.T, dQ)
        dW_k = np.dot(self.x.T, dK)
        dW_v = np.dot(self.x.T, dV)
    
        dx_q = np.dot(dQ, self.q_SA.T)
        dx_k = np.dot(dK, self.k_SA.T)
        dx_v = np.dot(dV, self.v_SA.T)
        dx = dx_q + dx_k + dx_v
    
        for i, idx in enumerate(self.input_indices):
            self.embeddings[idx] -= self.lr * dx[i]
    
        self.outt -= self.lr * dW_out
        self.q_SA -= self.lr * dW_q
        self.k_SA -= self.lr * dW_k
        self.v_SA -= self.lr * dW_v

    def train_step(self, input_indices, target_index):
        self.forward(input_indices)
        self.backward(target_index)

    def predict(self, input_indices):
        probs = self.forward(input_indices)
        return np.argmax(probs)

Создаем объект полученного класса с функциями преобразования текста

In [664]:
Tt = Text_transformation()

In [666]:
text = 'Когда человек сознательно или интуитивно выбирает себе в жизни какую-то цель, жизненную задачу, он невольно дает себе оценку. По тому, ради чего человек живет, можно судить и о его самооценке - низкой или высокой. Если человек живет, чтобы приносить людям добро, облегчать их страдания, давать людям радость, то он оценивает себя на уровне этой своей человечности. Он ставит себе цель, достойную человека. Только такая цель позволяет человеку прожить свою жизнь с достоинством и получить настоящую радость. Да, радость! Подумайте: если человек ставит себе задачей увеличивать в жизни добро, приносить людям счастье, какие неудачи могут его постигнуть? Не тому помочь? Но много ли людей не нуждаются в помощи? Если жить только для себя, своими мелкими заботами о собственном благополучии, то от прожитого не останется и следа. Если же жить для других, то другие сберегут то, чему служил, чему отдавал силы. Можно по-разному определять цель своего существования, но цель должна быть. Надо иметь и принципы в жизни. Одно правило в жизни должно быть у каждого человека, в его цели жизни, в его принципах жизни, в его поведении: надо прожить жизнь с достоинством, чтобы не стыдно было вспоминать. Достоинство требует доброты, великодушия, умения не быть эгоистом, быть правдивым, хорошим другом, находить радость в помощи другим.'

partt = [
    "Если человек живет, чтобы приносить людям",
    "Он ставит себе цель, достойную",
    "Только такая цель позволяет человеку прожить свою жизнь с",
    "Если жить только для себя, своими мелкими заботами о",
    "Если же жить для других, то другие сберегут то, чему",
    "Надо иметь и",
    "Одно правило в жизни должно быть у каждого",
    "Достоинство требует",
]

res = [
    "добро",
    "человека",
    "достоинством",
    "благополучии",
    "служил",
    "принципы",
    "человека",
    "доброты",
]

In [668]:
tokens = Tt.remove_stop_words(Tt.lemmas(Tt.tokenize(text)))

# Векторизация словаря
vocab = list(set(tokens))
word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}

In [670]:
X = []
y = []

Предсказание последнего слова

In [673]:
for phrase, label in zip(partt, res):
    tokens = Tt.remove_stop_words(Tt.lemmas(Tt.tokenize(phrase)))
    if len(tokens) < 2: continue
    input_indices = [word_to_index.get(w, 0) for w in tokens if w in word_to_index]
    label_index = word_to_index.get(label, 0)
    X.append(input_indices)
    y.append(label_index)


Обучение

In [675]:
model = Transformer(size=len(vocab), embed=16, hidden=32)

for epoch in range(5000):
    total_loss = 0
    for i in range(len(X)):
        model.train_step(X[i], y[i])
        probs = model.forward(X[i])
        total_loss += -np.log(probs[y[i]] + 1e-9)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss / len(X)}")


Epoch 0, Loss: 4.304065021993404
Epoch 100, Loss: 4.3040649576104695
Epoch 200, Loss: 4.304064893088274
Epoch 300, Loss: 4.304064828418661
Epoch 400, Loss: 4.304064763593434
Epoch 500, Loss: 4.304064698604365
Epoch 600, Loss: 4.30406463344318
Epoch 700, Loss: 4.304064568101564
Epoch 800, Loss: 4.304064502571164
Epoch 900, Loss: 4.304064436843572
Epoch 1000, Loss: 4.304064370910341
Epoch 1100, Loss: 4.304064304762967
Epoch 1200, Loss: 4.304064238392899
Epoch 1300, Loss: 4.304064171791529
Epoch 1400, Loss: 4.3040641049501955
Epoch 1500, Loss: 4.30406403786018
Epoch 1600, Loss: 4.304063970512699
Epoch 1700, Loss: 4.304063902898912
Epoch 1800, Loss: 4.304063835009912
Epoch 1900, Loss: 4.304063766836727
Epoch 2000, Loss: 4.304063698370312
Epoch 2100, Loss: 4.304063629601559
Epoch 2200, Loss: 4.304063560521278
Epoch 2300, Loss: 4.304063491120211
Epoch 2400, Loss: 4.304063421389018
Epoch 2500, Loss: 4.304063351318281
Epoch 2600, Loss: 4.304063280898498
Epoch 2700, Loss: 4.304063210120086
Epoc

Предсказание

In [678]:
print("Predictions:")
for i in partt:
    tokens = Tt.remove_stop_words(Tt.lemmas(Tt.tokenize(i)))
    input_indices = [word_to_index.get(w, 0) for w in tokens if w in word_to_index]
    pred_index = model.predict(input_indices)
    print(f"{i} -> {index_to_word[pred_index]}")

Predictions:
Если человек живет, чтобы приносить людям -> правдивый
Он ставит себе цель, достойную -> выбирать
Только такая цель позволяет человеку прожить свою жизнь с -> прожить
Если жить только для себя, своими мелкими заботами о -> радость
Если же жить для других, то другие сберегут то, чему -> задача
Надо иметь и -> определять
Одно правило в жизни должно быть у каждого -> радость
Достоинство требует -> сила
